# Intro
- Goal
- Hypothesis
- Data cycle
- Methods we'll use
- Significance

In [53]:
# Importing required libraries
import pandas as pd
import matplotlib.pyplot as plt

# Add prose about each library that is used

In [57]:
# Import CSV to pandas dataframe
df = pd.read_csv("data.csv")
df.head()

# Add prose about how we retreived the data and its source

,Bene_Geo_Lvl,Bene_Geo_Desc,Bene_Geo_Cd,Bene_Age_Lvl,Bene_Demo_Lvl,Bene_Demo_Desc,Bene_Cond,Prvlnc,Tot_Mdcr_Stdzd_Pymt_PC,Tot_Mdcr_Pymt_PC,Hosp_Readmsn_Rate,ER_Visits_Per_1000_Benes
0,State,Alabama,1.0,All,All,All,Alcohol Abuse,0.0188,25102.3405,23348.6039,0.2413,2184.7557
1,State,Alabama,1.0,65+,Dual Status,Medicare Only,Alcohol Abuse,0.0118,NaN,NaN,NaN,NaN
2,State,Alabama,1.0,<65,Dual Status,Medicare Only,Alcohol Abuse,0.0320,NaN,NaN,NaN,NaN
3,State,Alabama,1.0,All,Dual Status,Medicare Only,Alcohol Abuse,0.0147,NaN,NaN,NaN,NaN
4,State,Alabama,1.0,65+,Dual Status,Medicare and Medicaid,Alcohol Abuse,0.0238,NaN,NaN,NaN,NaN


In [58]:
# Cleaning the data

# Deleting unneeded columns
df.drop(columns = 'Bene_Geo_Cd', inplace = True)

# Renaming columns
df = df.set_axis(['Geo_Lvl', 'Geo_Desc', 'Age_Lvl', 'Feature', 'Feature_Desc', 
                     'Condition', 'Prevalence', 'Standardized_Medicare_Payment', 
                     'Total_Medicare_Payment', 'Hospital_Rate', 'ER_Rate_Per_1000'], 
                     axis = 1)

# Deleting rows at end with too much missing data
df.drop(df.tail(21).index, inplace = True)

df.head()

,Geo_Lvl,Geo_Desc,Age_Lvl,Feature,Feature_Desc,Condition,Prevalence,Standardized_Medicare_Payment,Total_Medicare_Payment,Hospital_Rate,ER_Rate_Per_1000
0,State,Alabama,All,All,All,Alcohol Abuse,0.0188,25102.3405,23348.6039,0.2413,2184.7557
1,State,Alabama,65+,Dual Status,Medicare Only,Alcohol Abuse,0.0118,NaN,NaN,NaN,NaN
2,State,Alabama,<65,Dual Status,Medicare Only,Alcohol Abuse,0.0320,NaN,NaN,NaN,NaN
3,State,Alabama,All,Dual Status,Medicare Only,Alcohol Abuse,0.0147,NaN,NaN,NaN,NaN
4,State,Alabama,65+,Dual Status,Medicare and Medicaid,Alcohol Abuse,0.0238,NaN,NaN,NaN,NaN
